In [1]:
import os
for dirname, _, filenames in os.walk('main'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
!pip install tensorflow
!pip install pandas
!pip install numpy
!pip install scipy
!pip install pillow
!pip install tqdm
!pip install virtualenv
!pip install scikit-learn

In [5]:
import pandas as pd
import numpy as np
import os
from PIL import Image
from tqdm.notebook import tqdm
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.image import load_img
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tqdm import tqdm

from skimage.transform import resize
from skimage.io import imread
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [6]:
main_dataset_path = "C:\\Users\\rupes\\OneDrive\\Desktop\\Palm\\Palm\\main"
anemiapath="C:\\Users\\rupes\\OneDrive\\Desktop\\Palm\\Palm\\main\\Anemic"
nonanemiapath="C:\\Users\\rupes\\OneDrive\\Desktop\\Palm\\Palm\\main\\Non_Anemic"

anemia_image_paths = [os.path.join(anemiapath, filename) for filename in tqdm(os.listdir(anemiapath))]
anemia_labels = [1] * len(anemia_image_paths)
#Anemic => 1
non_anemia_image_paths = [os.path.join(nonanemiapath, filename) for filename in tqdm(os.listdir(nonanemiapath))]
non_anemia_labels = [0] * len(non_anemia_image_paths)
#Non Anemic => 0
image_paths = anemia_image_paths + non_anemia_image_paths
labels = anemia_labels + non_anemia_labels

df = pd.DataFrame({"Image": image_paths, "Label": labels})


100%|██████████████████████████████████████████████████████████████████████████| 1698/1698 [00:00<00:00, 314295.15it/s]


In [7]:
Categories=['Anemic','Non_Anemic']
flat_data_arr=[] #input array
target_arr=[] #output array
datadir= "C:\\Users\\rupes\\OneDrive\\Desktop\\Palm\\Palm\\main"
#path which contains all the categories of images
for i in Categories:

    print(f'loading... category : {i}')
    path=os.path.join(datadir,i)
    for img in os.listdir(path):
        img_array=imread(os.path.join(path,img))
        img_resized=resize(img_array,(224,224,3))
        flat_data_arr.append(img_resized.flatten())
        target_arr.append(Categories.index(i))
    print(f'loaded category:{i} successfully')
flat_data=np.array(flat_data_arr)
target=np.array(target_arr)

loading... category : Anemic
loaded category:Anemic successfully
loading... category : Non_Anemic
loaded category:Non_Anemic successfully


In [8]:
flat_data
target=np.array(target_arr)
target

array([0, 0, 0, ..., 1, 1, 1])

In [9]:
df1=pd.DataFrame(flat_data)
df1

,0,1,2,3,4,5,6,7,8,9,...,150518,150519,150520,150521,150522,150523,150524,150525,150526,150527
0,0.953618,0.841553,0.965359,0.953948,0.841553,0.965359,0.955433,0.841553,0.965359,0.956541,...,0.898693,0.652941,0.478431,0.898693,0.652941,0.478431,0.898693,0.652941,0.478431,0.898693
1,0.842793,0.686113,0.935652,0.841290,0.685103,0.935467,0.837418,0.682352,0.934967,0.835216,...,0.935519,0.856180,0.690117,0.935921,0.856209,0.690196,0.935948,0.856209,0.690196,0.935948
2,0.842793,0.686113,0.935652,0.841290,0.685103,0.935467,0.837418,0.682352,0.934967,0.835216,...,0.935519,0.856180,0.690117,0.935921,0.856209,0.690196,0.935948,0.856209,0.690196,0.935948
3,0.856209,0.690196,0.935948,0.856209,0.690196,0.935948,0.856180,0.690117,0.935921,0.855740,...,0.934338,0.837418,0.682352,0.934967,0.841290,0.685103,0.935467,0.842793,0.686113,0.935652
4,0.652941,0.478431,0.898693,0.652941,0.478431,0.898693,0.652941,0.478431,0.898693,0.652941,...,0.965053,0.955433,0.841553,0.965359,0.953948,0.841553,0.965359,0.953618,0.841553,0.965359
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4255,0.516823,0.293409,0.871688,0.507465,0.281530,0.869288,0.497886,0.270797,0.867306,0.493306,...,0.923347,0.755596,0.596119,0.923536,0.753381,0.594378,0.923325,0.749065,0.591202,0.922986
4256,0.681110,0.490358,0.903917,0.679277,0.489150,0.903515,0.678420,0.488585,0.903326,0.679706,...,0.909727,0.704034,0.521681,0.911130,0.719359,0.537056,0.913701,0.719574,0.537380,0.913773
4257,0.681110,0.490358,0.903917,0.679277,0.489150,0.903515,0.678420,0.488585,0.903326,0.679706,...,0.909727,0.704034,0.521681,0.911130,0.719359,0.537056,0.913701,0.719574,0.537380,0.913773
4258,0.681110,0.490358,0.903917,0.679277,0.489150,0.903515,0.678420,0.488585,0.903326,0.679706,...,0.909727,0.704034,0.521681,0.911130,0.719359,0.537056,0.913701,0.719574,0.537380,0.913773


In [10]:
df1["Target"]=target
df1

,0,1,2,3,4,5,6,7,8,9,...,150519,150520,150521,150522,150523,150524,150525,150526,150527,Target
0,0.953618,0.841553,0.965359,0.953948,0.841553,0.965359,0.955433,0.841553,0.965359,0.956541,...,0.652941,0.478431,0.898693,0.652941,0.478431,0.898693,0.652941,0.478431,0.898693,0
1,0.842793,0.686113,0.935652,0.841290,0.685103,0.935467,0.837418,0.682352,0.934967,0.835216,...,0.856180,0.690117,0.935921,0.856209,0.690196,0.935948,0.856209,0.690196,0.935948,0
2,0.842793,0.686113,0.935652,0.841290,0.685103,0.935467,0.837418,0.682352,0.934967,0.835216,...,0.856180,0.690117,0.935921,0.856209,0.690196,0.935948,0.856209,0.690196,0.935948,0
3,0.856209,0.690196,0.935948,0.856209,0.690196,0.935948,0.856180,0.690117,0.935921,0.855740,...,0.837418,0.682352,0.934967,0.841290,0.685103,0.935467,0.842793,0.686113,0.935652,0
4,0.652941,0.478431,0.898693,0.652941,0.478431,0.898693,0.652941,0.478431,0.898693,0.652941,...,0.955433,0.841553,0.965359,0.953948,0.841553,0.965359,0.953618,0.841553,0.965359,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4255,0.516823,0.293409,0.871688,0.507465,0.281530,0.869288,0.497886,0.270797,0.867306,0.493306,...,0.755596,0.596119,0.923536,0.753381,0.594378,0.923325,0.749065,0.591202,0.922986,1
4256,0.681110,0.490358,0.903917,0.679277,0.489150,0.903515,0.678420,0.488585,0.903326,0.679706,...,0.704034,0.521681,0.911130,0.719359,0.537056,0.913701,0.719574,0.537380,0.913773,1
4257,0.681110,0.490358,0.903917,0.679277,0.489150,0.903515,0.678420,0.488585,0.903326,0.679706,...,0.704034,0.521681,0.911130,0.719359,0.537056,0.913701,0.719574,0.537380,0.913773,1
4258,0.681110,0.490358,0.903917,0.679277,0.489150,0.903515,0.678420,0.488585,0.903326,0.679706,...,0.704034,0.521681,0.911130,0.719359,0.537056,0.913701,0.719574,0.537380,0.913773,1


In [11]:
#input data
x=df1.iloc[:,:-1]
#output data
y=df1.iloc[:,-1]

In [12]:
x

,0,1,2,3,4,5,6,7,8,9,...,150518,150519,150520,150521,150522,150523,150524,150525,150526,150527
0,0.953618,0.841553,0.965359,0.953948,0.841553,0.965359,0.955433,0.841553,0.965359,0.956541,...,0.898693,0.652941,0.478431,0.898693,0.652941,0.478431,0.898693,0.652941,0.478431,0.898693
1,0.842793,0.686113,0.935652,0.841290,0.685103,0.935467,0.837418,0.682352,0.934967,0.835216,...,0.935519,0.856180,0.690117,0.935921,0.856209,0.690196,0.935948,0.856209,0.690196,0.935948
2,0.842793,0.686113,0.935652,0.841290,0.685103,0.935467,0.837418,0.682352,0.934967,0.835216,...,0.935519,0.856180,0.690117,0.935921,0.856209,0.690196,0.935948,0.856209,0.690196,0.935948
3,0.856209,0.690196,0.935948,0.856209,0.690196,0.935948,0.856180,0.690117,0.935921,0.855740,...,0.934338,0.837418,0.682352,0.934967,0.841290,0.685103,0.935467,0.842793,0.686113,0.935652
4,0.652941,0.478431,0.898693,0.652941,0.478431,0.898693,0.652941,0.478431,0.898693,0.652941,...,0.965053,0.955433,0.841553,0.965359,0.953948,0.841553,0.965359,0.953618,0.841553,0.965359
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4255,0.516823,0.293409,0.871688,0.507465,0.281530,0.869288,0.497886,0.270797,0.867306,0.493306,...,0.923347,0.755596,0.596119,0.923536,0.753381,0.594378,0.923325,0.749065,0.591202,0.922986
4256,0.681110,0.490358,0.903917,0.679277,0.489150,0.903515,0.678420,0.488585,0.903326,0.679706,...,0.909727,0.704034,0.521681,0.911130,0.719359,0.537056,0.913701,0.719574,0.537380,0.913773
4257,0.681110,0.490358,0.903917,0.679277,0.489150,0.903515,0.678420,0.488585,0.903326,0.679706,...,0.909727,0.704034,0.521681,0.911130,0.719359,0.537056,0.913701,0.719574,0.537380,0.913773
4258,0.681110,0.490358,0.903917,0.679277,0.489150,0.903515,0.678420,0.488585,0.903326,0.679706,...,0.909727,0.704034,0.521681,0.911130,0.719359,0.537056,0.913701,0.719574,0.537380,0.913773


In [13]:
y

0       0
1       0
2       0
3       0
4       0
       ..
4255    1
4256    1
4257    1
4258    1
4259    1
Name: Target, Length: 4260, dtype: int32

In [14]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=77,stratify=y)

In [15]:
# param_grid={'C':[0.1],
#             'gamma':[0.0001],
#             'kernel':['rbf','poly']}
# svc=svm.SVC(probability=True)
# model=GridSearchCV(svc,param_grid)
# model.fit(x_train,y_train)

svm = SVC(kernel="rbf", gamma=0.5, C=1.0)
svm.fit(x_train, y_train)
y_pred=svm.predict(x_test)
accuracy=accuracy_score(y_pred,y_test)
print(f"The model is {accuracy*100}% accurate")
print(classification_report(y_test, y_pred, target_names=['Anemic_jpeg', 'Non_Anemic_jpeg']))

The model is 88.73239436619718% accurate
                 precision    recall  f1-score   support

    Anemic_jpeg       0.84      1.00      0.91       512
Non_Anemic_jpeg       1.00      0.72      0.84       340

       accuracy                           0.89       852
      macro avg       0.92      0.86      0.87       852
   weighted avg       0.91      0.89      0.88       852

